In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from datasets import load_dataset

# hyperparameters
batch_size = 8 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

df = load_dataset("Teejeigh/raw_friends_series_transcript")["train"]["text"]

/home/stef/anaconda3/envs/mnlp-p2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
text = "\n".join(df)
chars = sorted((set(text)))
vocab_size = len(chars)

In [3]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [4]:
decode(encode("hello")) # tokenization 

'hello'

In [21]:

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # C is vocab dim! T is the number of tokens in the context for example the context is "hello" then T=5
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

In [22]:
model = BigramLanguageModel(vocab_size)

out = model.token_embedding_table(torch.tensor(encode("stef")))

out = out.view(-1, vocab_size)
out.shape

torch.Size([4, 91])

In [23]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [24]:
train_data.shape

torch.Size([3604275])

In [ ]:
block_size = 8 # what is the maximum context length for predictions?

In [26]:
def get_sample(data, i):
    seq_len = min(block_size, len(data) - 1 - i)
    inputs = data[i:i+seq_len]
    targets = data[i+1:i+1+seq_len].view(-1)
    return inputs, targets

In [ ]:
epochs = 10000 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(epochs):
    rand = torch.randint(0, len(train_data) - block_size, (1,)) 
    inputs, targets = get_sample(train_data, rand[0])
    out, loss = model(inputs.unsqueeze(0), targets.unsqueeze(0))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss)

tensor(3.3665, grad_fn=<NllLossBackward0>)


In [33]:
in1 = "Rache"

out, _ = model(torch.tensor(encode(in1)).unsqueeze(0)) #

# take max 
out = out.view(-1, vocab_size) 
preds = torch.argmax(out, dim=1)


In [40]:
decode(preds.tolist())

'onhe '

In [ ]:
# vocab x vocab matrix ; iterate over dataset, i store in freq matrix the count of each bigram

In [ ]:
# TODO: implement LSTM to work to predict the next token from 10
# make dataset in sequences of 10 characters, and predict the next character 
seq_len = 10
def sequence_data(data, i): # 8 block size
    inputs = data[i:i+seq_len]
    targets = data[i+1:i+1+seq_len].view(-1)
    return inputs, targets
# DECODER  
class LSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size) # 'stef' -> [0, 1, 2, 3] -> [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6], [0.7, 0.8]]
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size) # predict the next token

    def forward(self, x, hidden=None):
        x = self.embedding(x)

        print(x.shape)
        x, hidden = self.lstm(x, hidden)
        # take lost token 
        # x = x[:, -1]
        # print(x.shape)
        x = self.fc(x)
        return x, hidden # return next token 

In [ ]:
# encoder stef saka <MASK> <MASK> livestock mnogu <MASK> a boko ide na salsa

# encoder - only models -> embeding models (BERT)
# decoder only (GPT)

In [61]:
sequence_data(train_data, 10)

(tensor([ 1, 54, 70, 67, 80, 67,  1, 44, 77, 76]),
 tensor([54, 70, 67, 80, 67,  1, 44, 77, 76, 71]))

In [ ]:
# make sequence dataset
import sys 

hidden_size = 16 

model = LSTM(vocab_size, hidden_size)

for i in range(epochs):
    rand = torch.randint(0, len(train_data) - seq_len, (1,)) 
    inputs, targets = sequence_data(train_data, rand[0])
    out, loss = model(inputs.unsqueeze(0), targets.unsqueeze(0))
    sys.exit()
    


tensor([77, 67, 87, 26,  1, 54, 67, 74, 74, 12])
torch.Size([10])
torch.Size([10])
torch.Size([1, 10, 16])


IndexError: index 1 is out of bounds for dimension 0 with size 1